load required modules

In [7]:
import streamlit as st
import pandas as pd
import os
from datetime import date, timedelta
#import matplotlib.pyplot as plt

set parameters

In [ ]:
leave_planner_data = "weekly_leave_multi.csv"
num_weeks = 52
year = 2025   # you can make this a user input if you want
decimals = 1 # number of decimal places to display

load base staff data

In [3]:
def load_staff_data(staff_base_data):
    # if os.path.exists(staff_base_data):
    df = pd.read_csv(staff_base_data)
    #df["week_commencing"] = pd.to_datetime(df["week_commencing"]).dt.date
    return df
    # else:
    #     df = create_empty(year, staff_list)
    #     df.to_csv(leave_planner_data, index=False)
    #     return df

staff_list = load_staff_data('staff_list.csv')

print(staff_list)

           staff_member job_role  hours_pw  leave_allowance_days
0     Attard, Caroline       TBC      37.5                    30
1       Banda, Maureen       TBC      37.5                    30
2      Baxendale, Vicki      TBC      37.5                    30
3       Brennan, Sarah       TBC      37.5                    30
4        Bristow, Simon      TBC      37.5                    30
5      Connell, Michael      TBC      37.5                    30
6        Gitsham, Nick       TBC      37.5                    30
7     Griffiths, Shaun       TBC      37.5                    30
8         Hopley, Paul       TBC      37.5                    30
9   Igbonwoke, Frances       TBC      37.5                    30
10     Mhlanga, Fortune      TBC      37.5                    30
11  Polychronakis, Andy      TBC      37.5                    30
12         Smit, Lizell      TBC      37.5                    30
13       Thomas, Louise      TBC      37.5                    30


get list of just staff names for use later

In [4]:
staff_names = staff_list['staff_member'].to_list()
print(staff_names)

['Attard, Caroline ', 'Banda, Maureen ', 'Baxendale, Vicki', 'Brennan, Sarah ', 'Bristow, Simon', 'Connell, Michael', 'Gitsham, Nick ', 'Griffiths, Shaun ', 'Hopley, Paul ', 'Igbonwoke, Frances ', 'Mhlanga, Fortune', 'Polychronakis, Andy', 'Smit, Lizell', 'Thomas, Louise']


function to load or create calendar and add any staff that are missing

In [8]:
def load_or_update_leave_file(filepath, staff_list):
    """
    Loads an existing weekly leave CSV OR creates/updates one
    with all weeks of the current year for all staff.

    Weeks are auto-generated based on today's date.
    """

    # -----------------------------------------
    # Auto-generate weekly start/end dates
    # -----------------------------------------
    today = date.today()
    year = today.year

    # First Monday of the year
    first_day = date(year, 1, 1)
    first_monday = first_day + timedelta(days=(7 - first_day.weekday()) % 7)

    # Last Monday of the year
    last_day = date(year, 12, 31)
    last_monday = last_day - timedelta(days=last_day.weekday())

    # Weekly list of Mondays
    weeks = pd.date_range(start=first_monday, end=last_monday, freq="W-MON")

    # Create full weekly structure
    def create_full_structure(staff):
        rows = []
        for s in staff:
            for w in weeks:
                rows.append({
                    "staff_name": s,
                    "week_commencing": w,
                    "week_number": w.isocalendar().week,
                    "days_leave": 0
                })
        return pd.DataFrame(rows)

    # -----------------------------------------
    # CASE 1: File exists → load + update
    # -----------------------------------------
    if os.path.exists(filepath):
        existing = pd.read_csv(filepath, parse_dates=["week_commencing"])

        existing_staff = set(existing["staff_name"].unique())
        new_staff = set(staff_list) - existing_staff

        if new_staff:
            add_df = create_full_structure(new_staff)
            updated = pd.concat([existing, add_df], ignore_index=True)
            updated.to_csv(filepath, index=False)
            return updated

        return existing

    # -----------------------------------------
    # CASE 2: File does not exist → create new file
    # -----------------------------------------
    df = create_full_structure(staff_list)
    df.to_csv(filepath, index=False)
    return df



test working of above

In [ ]:
leave_calendar_df = load_or_update_leave_file('annual_leave_calendar.csv',staff_names)
#print(leave_calendar_df)

            staff_name week_commencing  week_number  days_leave
0    Attard, Caroline       2025-01-06            2           0
1    Attard, Caroline       2025-01-13            3           0
2    Attard, Caroline       2025-01-20            4           0
3    Attard, Caroline       2025-01-27            5           0
4    Attard, Caroline       2025-02-03            6           0
..                 ...             ...          ...         ...
723     Thomas, Louise      2025-12-01           49           0
724     Thomas, Louise      2025-12-08           50           0
725     Thomas, Louise      2025-12-15           51           0
726     Thomas, Louise      2025-12-22           52           0
727     Thomas, Louise      2025-12-29            1           0

[728 rows x 4 columns]
